In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import datetime

torch.set_printoptions(edgeitems=2)
# 默认当前数据处理
torch.manual_seed(123)

In [6]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

data_path = '../data-unversioned/p1ch7/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Files already downloaded and verified
Files already downloaded and verified


In [9]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        # 这里使用torch.tanh 方法，需要替换成
        # out = torch.tanh(self.fc1(out))
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        for imgs, labels in train_loader:  # <3>
            outputs = model(imgs)  # <4>
            loss = loss_fn(outputs, labels)  # <5>
            optimizer.zero_grad()  # <6>
            loss.backward()  # <7>
            optimizer.step()  # <8>
            loss_train += loss.item()  # <9>
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # <10>


In [10]:
model1 = Net()
numel_list = [p.numel() for p in model1.parameters()]
sum(numel_list), numel_list

(18354, [432, 16, 1152, 8, 16384, 32, 320, 10])

In [7]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)  # <1>
model = Net()  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2023-05-22 17:15:46.214052 Epoch 1, Training loss 2.064554848481932
2023-05-22 17:19:21.931063 Epoch 10, Training loss 1.106541002650395
2023-05-22 17:23:36.275285 Epoch 20, Training loss 0.9395636208831807
2023-05-22 17:27:43.153742 Epoch 30, Training loss 0.8512452320979379
2023-05-22 17:31:56.978622 Epoch 40, Training loss 0.7930895810008354
2023-05-22 17:36:06.462438 Epoch 50, Training loss 0.7549725945876993
2023-05-22 17:40:29.613319 Epoch 60, Training loss 0.7197639750855048
2023-05-22 17:44:41.653830 Epoch 70, Training loss 0.6972395716725713
2023-05-22 17:49:01.887512 Epoch 80, Training loss 0.677836147377558
2023-05-22 17:53:08.739394 Epoch 90, Training loss 0.6626793948357063
2023-05-22 17:57:35.824146 Epoch 100, Training loss 0.6401451705666759


In [8]:
# 使用loader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.77
Accuracy val: 0.66
